# Transfer Learning is  a technique used to get better accuracy. Almost state of the art result.

In [1]:
""" Here I Will be taking a simple Dataset which is CAT vs DOG, But YOu can try to take any dataset for practical"""

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-07-29 03:23:44--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.234.128, 108.177.112.128, 74.125.124.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.234.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   127MB/s    in 0.5s    

2020-07-29 03:23:45 (127 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
""" If you are using Google COlab, YOu will find this downlaoded dataset into your GOogle Drive. Check it

    It is downloaded in the zip mode, We will unzip it later 
    
     Meanwhile I Will import the libraries """

  
import zipfile
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub


In [3]:
""" Unzipping the Dataset which I Downloaded """

dataset_path = "./cats_and_dogs_filtered.zip"

zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

zip_object.extractall("./")

zip_object.close()

In [4]:
""" Setting up the Dataset path """

dataset_path_new = "./cats_and_dogs_filtered/"

train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

# Building the Model, Using a Pre-Trained Model

MOBIle Net V2

In [6]:
IMG_SHAPE = (128, 128, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

""" Parameter illustration 

1. Input_shape : It accepts your pixel size and channel
2. include_Top : False , It means Do you want to include the Top layer or not, False means NO i do not want that
3. weights = Imagenet, This is the Imagenet model weights"""

9412608/9406464 [==============================] - 0s 0us/step


' Parameter illustration \n\n1. Input_shape : It accepts your pixel size and channel\n2. include_Top : False , It means Do you want to include the Top layer or not, False means NO i do not want that\n3. weights = Imagenet, This is the Imagenet'

In [7]:
""" FREEZING THE MODEL """

base_model.trainable = False

""" Designing  the custom head of the model """

base_model.output

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output) #Adding this layer


""" Adding a final layer, Which will be OutPut Layer with a sigmoid function

Activation function is Sigmoid """

prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

""" Combining the custom head of the model and pre-trained model into one final model """

model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)




# Compiling the model

In [8]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

# Creating Data Generator

In [9]:
""" Resizing the image because this MobileNet support only few shapes of images like 96, 96), (128, 128), (160, 160), (192, 192), (224, 224)"""


""" Performing the Normalization """

data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

""" Using DataGenerator function """

train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")



Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Training the Model

In [10]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
16/16 [==============================] - 11s 695ms/step - loss: 0.7120 - accuracy: 0.6055 - val_loss: 0.6236 - val_accuracy: 0.6790
Epoch 2/5
16/16 [==============================] - 10s 600ms/step - loss: 0.5966 - accuracy: 0.6950 - val_loss: 0.5396 - val_accuracy: 0.7480
Epoch 3/5
16/16 [==============================] - 9s 593ms/step - loss: 0.5190 - accuracy: 0.7545 - val_loss: 0.4727 - val_accuracy: 0.7960
Epoch 4/5
16/16 [==============================] - 9s 583ms/step - loss: 0.4571 - accuracy: 0.8060 - val_loss: 0.4168 - val_accuracy: 0.8380
Epoch 5/5
16/16 [==============================] - 9s 592ms/step - loss: 0.4036 - accuracy: 0.8425 - val_loss: 0.3694 - val_accuracy: 0.8590


# Transfering Learning Model Evaluation

In [11]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

print("Accuracy after transfer learning: {}".format(valid_accuracy))

Instructions for updating:
Please use Model.evaluate, which supports generators.
Accuracy after transfer learning: 0.859000027179718


# Transfering Learning and Fine Tuning the Model 

### Transfer learning 

consists of taking features learned on one problem, and leveraging them on a new, similar problem. For instance, features from a model that has learned to identify racoons may be useful to kick-start a model meant to identify tanukis.

Transfer learning is usually done for tasks where your dataset has too little data to train a full-scale model from scratch.


### Fine Tuning Model

Fine-tuning, which consists of unfreezing the entire model you obtained above (or part of it), and re-training it on the new data with a very low learning rate. This can potentially achieve meaningful improvements, by incrementally adapting the pretrained features to the new data.

Fine tuning should only be performed After transfer learning. It improves on the previous result



NOTE : If we try to perform fine tuning immediately in our custom head is not trained.
gradients will be much different between custom had and our unfrozen layers from the base network to


In [12]:
""" Unfreezing few Top Layers """

base_model.trainable = True

print("Number of layersin the base model: {}".format(len(base_model.layers)))



""" This will store the Index on which layer, We are going to fine tune , Here I am taking fine_Tune at 100, 

I will fine tune all the layers from 100 to 155, 155 is the end of the layers"""

fine_tune_at = 100



for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

Number of layersin the base model: 155


# Compiling the Model for Fine Tuning 



In [13]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
""" Fine Tuning the Model """

model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 11s 675ms/step - loss: 0.1973 - accuracy: 0.9240 - val_loss: 0.0838 - val_accuracy: 0.9690
Epoch 2/5
16/16 [==============================] - 10s 614ms/step - loss: 0.0256 - accuracy: 0.9990 - val_loss: 0.0846 - val_accuracy: 0.9700
Epoch 3/5
16/16 [==============================] - 10s 604ms/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.1233 - val_accuracy: 0.9610
Epoch 4/5
16/16 [==============================] - 10s 632ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.1073 - val_accuracy: 0.9690
Epoch 5/5
16/16 [==============================] - 10s 639ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.1103 - val_accuracy: 0.9700


In [15]:
""" Evaluation of the Fine Tune Model """

valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9700000286102295
